# Titanic Dataset
#### Applying Machine Learning methods to predict the survivability status of passengers
##### @author: Amitav Mostafa, Younes Eloiarm

## Initialize dataset

In [1]:
#Import the important packages
import pandas as pd
import numpy as np
import os, datetime

#Models
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

#Validation Fix
from sklearn.model_selection import train_test_split

#Model Quality Fix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
# for root mean squared error rmse
#rms = mean_squared_error(y, X, squared= False)
from sklearn.model_selection import cross_val_score

#Bundling Fix
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#NaN Fix
from sklearn.impute import SimpleImputer

#Categorical/Object Data Fix
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

#Plotting
import matplotlib.pyplot as plt
%matplotlib inline

/Users/amitavchrismostafa/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/amitavchrismostafa/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/amitavchrismostafa/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/amitavchrismostafa/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [5]:
cwd = os.getcwd() # get current working directory

path_data = cwd + '/titanic/' #set path to data files

data_test = pd.read_csv(path_data+'test.csv')
data_train = pd.read_csv(path_data+'train.csv') #load datasets

'/Users/amitavchrismostafa/Documents/Python/titanic-ml'

In [7]:
data_train.head() # display training data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
data_test.head() # display testing data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Clean up and process the datasets

In [13]:
print(data_test.isnull().any()) #Inspect existence of NaNs in the datasets
print(data_train.isnull().any())

PassengerId    False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare            True
Cabin           True
Embarked       False
dtype: bool
PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool


As we can observe above, our target column 'Survived' has no missing values 

In [ ]:
#Get and Display current working directory
cwd = os.getcwd()

#Change Directory to parent (one step up)
filepath_elements = cwd.split('/')
filepath_elements.pop()
filepath= '/'.join(filepath_elements)
filepath

#Save filenames and path as str in variables
datapath = '/30-days-of-ml/'
dataname_test = 'test.csv'
dataname_train = 'train.csv'
dataname_samplesubmit = 'sample_submission.csv'

#Read the csv files and initialize as Dataframes
data_test = pd.read_csv(filepath+datapath+dataname_test, index_col='id')
data_train = pd.read_csv(filepath+datapath+dataname_train, index_col='id')
data_samplesubmit = pd.read_csv(filepath+datapath+dataname_samplesubmit, index_col='id')

#Display Data for viewing
data_samplesubmit.head(10);
data_train.head(10);
data_test.head(10);

#Getting Columns
train_cols = list(data_train.columns)
print(train_cols);
test_cols = list(data_test.columns)
print(test_cols);
train_cats = [col for col in train_cols if 'cat' in col]
train_conts = [col for col in train_cols if 'cont' in col]
test_cats = [col for col in test_cols if 'cat' in col]
test_conts = [col for col in test_cols if 'cont' in col]

#Inspect for NaNs
print(data_train.isnull().sum().sum())
data_train.isnull().any();
print(data_test.isnull().sum().sum())
data_test.isnull().any();

#Inspect categorical data for cardinality
cardinals_train = list(map(lambda col: data_train[col].nunique(), train_cats))
save_cardinals_train = dict(zip(train_cats, cardinals_train))
sorted(save_cardinals_train.items(), key=lambda x: x[1]);

cardinals_test = list(map(lambda col: data_test[col].nunique(), test_cats))
save_cardinals_test = dict(zip(train_cats, cardinals_test))
sorted(save_cardinals_test.items(), key=lambda x: x[1]);

#filter low cardinality columns
filter_limit = 10

low_cardinality_categorical_cols = [col for col in data_train.columns if data_train[col].nunique()<filter_limit and data_train[col].dtype == 'object']

#filter only numerical columns
numerical_cols = train_conts.copy()

#get columns to be used 
selected_cols = low_cardinality_categorical_cols + numerical_cols


In [ ]:
#Get full X and full y from data_train and thus X_test from data_test

y_full = data_train.target.copy()

X_full = data_train[selected_cols].copy()

X_test = data_test[selected_cols].copy()

#X_train, X_valid, y_train, y_valid = train_test_split(X_full, y_full, train_size=0.8, test_size=0.2,
#                                                                random_state=0)

In [ ]:
#Make your pipelines

#Preprocessing for numerical data; no NaNs in data means not used though
numerical_transformer = SimpleImputer(strategy='constant')

numerical_stategies = ['constant', 'mean', 'median', 'most_frequent']

#Preprocessing for categorical data

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

categorical_strategies = ['most_frequent']

#Bundle Preprocessing together => numerical and categorical 

preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, low_cardinality_categorical_cols)])

In [ ]:
#Make your pipelines

#Preprocessing for numerical data; no NaNs in data means not used though
numerical_transformer = SimpleImputer(strategy='constant')

numerical_stategies = ['constant', 'mean', 'median', 'most_frequent']

#Preprocessing for categorical data

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

categorical_strategies = ['most_frequent']

#Bundle Preprocessing together => numerical and categorical 

preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, low_cardinality_categorical_cols)])

In [ ]:
#Select best approach; Generate test predictions

final_dtr = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', DecisionTreeRegressor(max_leaf_nodes=max_nodes, random_state=1))
    ])

final_dtr.fit(y_full, X_full)
predict_dtr = final_dtr.predict(X_test)

output_dtr = pd.DataFrame ( {'Id':X_test.index, 'target':predict_test_dtr})
output_dtr.to_csv('submission_dtr.csv', index=False)
